In [ ]:
pip install langchain sentence-transformers chromadb llama-cpp-python langchain-community pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 27.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/

# Importing Libraries

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [ ]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Bio Mistral Medical RAG Chatbhot/Data")
docs = loader.load()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
len(docs)

198

In [ ]:
docs[100]

Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2013-04-02T15:48:18+01:00', 'author': 'ULStudent:JOHN.ALLEN', 'moddate': '2013-04-02T15:48:18+01:00', 'source': '/content/drive/MyDrive/Bio Mistral Medical RAG Chatbhot/Data/Oncology.pdf', 'total_pages': 149, 'page': 100, 'page_label': '101'}, page_content='101 \n \nChapter 8: Breast Reconstruction \nIntroduction \nBreast reconstruction surgery rebuilds the breast mound so that it is about  the same size and \nshape as before. The nipple and the a reola can also be added. Most women who ha ve had a \nmastectomy can have reconstruction. Women who have had a lumpectomy may not need \nreconstruction (www.breastcancer.org). \n \nImplants \nTissue expansion followed by removal of the expander and placement of a permanent implant \nis the most common form of reconstruction in the US. Implants do not cause ptosis (droop) \nbut are more suited for wom en with small to modera

#Chunking

In [ ]:
Text_Splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)
chunks = Text_Splitter.split_documents(docs)

In [ ]:
len(chunks)

1379

In [ ]:
chunks[777]

Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2013-04-02T15:48:18+01:00', 'author': 'ULStudent:JOHN.ALLEN', 'moddate': '2013-04-02T15:48:18+01:00', 'source': '/content/drive/MyDrive/Bio Mistral Medical RAG Chatbhot/Data/Oncology.pdf', 'total_pages': 149, 'page': 120, 'page_label': '121'}, page_content='121 \n \n3) Denial  \nDenial is a common reaction  following cancer diagnosis, especially when life threatening. \nAfter being told their diagnosis, approximately 20% of patients  deny they have cancer; 26% \npartially suppress awareness of implementing death and 8% demonstrate complete denial')

# Embeddings Creation

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""


In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name= "NeuML/pubmedbert-base-embeddings")

/tmp/ipython-input-11-1452316867.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name= "NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Vector Store Creation

In [ ]:
vectorstore = Chroma.from_documents(chunks,embeddings)

In [ ]:
query = "Who is at the risk of HIV?"

search_results = vectorstore.similarity_search(query)

In [ ]:
search_results

In [ ]:
retriver = vectorstore.as_retriever(search_kwargs={'k':5})

In [ ]:
retriver.get_relevant_documents(query)

#LLM Model Loading

In [ ]:
llm = LlamaCpp(
    model_path="/content/drive/MyDrive/Bio Mistral Medical RAG Chatbhot/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048,
    top_p=1,
    n_ctx=4096,
    verbose=True
)


# Use LLM and retriver and query, to generate final response

In [ ]:
template = """
<context>
You are an Medical Assistant that follows the instructions and generate the accurate response based on the query and the context provided.
Please be truthful and give direct answers.
</s>
<user>
{query}
</s>
<assistant>
"""


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate


In [ ]:
prompt = ChatPromptTemplate.from_template(template)


In [ ]:
rag_chain = (
    {"context": retriver, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
response = rag_chain.invoke(query)

In [ ]:
response

In [ ]:
import sys

while True:
    user_input = input(f"Input query: ")
    if user_input == 'exit':
        print("Exiting...")
        sys.exit()
    if user_input == "":
        continue
    result = rag_chain.invoke(user_input)
    print("Answer: ", result)
